In [2]:
import sys
import os

# Go up one directory (from notebooks/ to OLIST/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to path: {project_root}")

Project root added to path: d:\My_Projects\OLIST\python


In [3]:
from src.utils import fetch_data_from_bq
import pandas as pd
from src import sql_queries as q
import pandas as pd
import numpy as np



In [4]:
df = fetch_data_from_bq(q.GET_FACT_ORDERS)
df

BigQuery clients initialized.
Query successful. Scanned 0.000 MB.


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value
0,809a282bbd5dbcabb6f2f724fca862ec,622e13439d6b5a0b486c435618b2679e,canceled,2016-09-13 15:24:19+00:00,2016-10-07 13:16:46+00:00,NaT,NaT,2016-09-30 00:00:00+00:00,1.0,credit_card,2.0,40.950000000
1,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04 21:15:19+00:00,2016-10-07 13:18:03+00:00,2016-10-18 13:14:51+00:00,NaT,2016-10-20 00:00:00+00:00,1.0,credit_card,1.0,136.230000000
2,e5215415bb6f76fe3b7cb68103a0d1c0,b6f6cbfc126f1ae6723fe2f9b3751208,canceled,2016-10-22 08:25:27+00:00,NaT,NaT,NaT,2016-10-24 00:00:00+00:00,1.0,voucher,1.0,61.990000000
3,71303d7e93b399f5bcd537d124c0bcfa,b106b360fe2ef8849fbbd056f777b4d5,canceled,2016-10-02 22:07:52+00:00,2016-10-06 15:50:56+00:00,NaT,NaT,2016-10-25 00:00:00+00:00,1.0,credit_card,1.0,109.340000000
4,3b697a20d9e427646d92567910af6d57,355077684019f7f60a031656bd7262b8,delivered,2016-10-03 09:44:50+00:00,2016-10-06 15:50:54+00:00,2016-10-23 14:02:13+00:00,2016-10-26 14:02:13+00:00,2016-10-27 00:00:00+00:00,1.0,boleto,1.0,45.460000000
...,...,...,...,...,...,...,...,...,...,...,...,...
99435,70c5ccc21360b2f9c8f0a0decfb4e41d,d4217fcb991f5bc95ee2f5d7fd903cad,delivered,2018-08-14 19:46:25+00:00,2018-08-14 19:55:29+00:00,2018-08-16 16:26:00+00:00,2018-08-29 19:02:39+00:00,2018-10-23 00:00:00+00:00,1.0,credit_card,5.0,67.140000000
99436,869997fbe01f39d184956b5c6bccfdbe,55c9dad94ec1a2ba57998bdb376c230a,canceled,2018-09-26 08:40:15+00:00,NaT,NaT,NaT,2018-10-25 00:00:00+00:00,1.0,voucher,1.0,94.630000000
99437,0ba4f7e89d304a02edca85087abfd14c,b5c1c56fe1ec3cec6893b98d90a339bd,delivered,2018-08-13 22:51:49+00:00,2018-08-13 23:04:09+00:00,2018-08-22 14:04:00+00:00,2018-08-30 23:38:26+00:00,2018-10-25 00:00:00+00:00,1.0,credit_card,10.0,870.040000000
99438,10a045cdf6a5650c21e9cfeb60384c16,a4b417188addbc05b26b72d5e44837a1,canceled,2018-10-17 17:30:18+00:00,NaT,NaT,NaT,2018-10-30 00:00:00+00:00,1.0,voucher,1.0,89.710000000


In [39]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from typing import List, Optional, Dict, Union

# Assuming sql_queries and fetch_data_from_bq are available
# from . import sql_queries as q
# from .utils import fetch_data_from_bq

# --- 1. The Math Engines (Detection Core - No change) ---

def detect_iqr_outliers(df: pd.DataFrame, col_name: str, factor: float = 1.5) -> pd.DataFrame:
    """Detects outliers using Interquartile Range (IQR). Useful for non-normal distributions."""
    if df.empty or col_name not in df.columns:
        return pd.DataFrame()
    data = df[col_name]
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (factor * iqr)
    upper_bound = q3 + (factor * iqr)
    outliers = df[(data < lower_bound) | (data > upper_bound)].copy()
    outliers['method'] = 'IQR'
    outliers['lower_limit'] = lower_bound
    outliers['upper_limit'] = upper_bound
    outliers['anomaly_type'] = np.where(outliers[col_name] > upper_bound, 'Peak (High)', 'Valley (Low)')
    return outliers

def detect_zscore_outliers(df: pd.DataFrame, col_name: str, threshold: float = 3.0) -> pd.DataFrame:
    """Standard Z-Score method. Best for normal distributions."""
    if df.empty or col_name not in df.columns:
        return pd.DataFrame()
    data = df[col_name]
    mean = data.mean()
    std = data.std()
    if std == 0: 
        return pd.DataFrame()
    z_scores = (data - mean) / std
    outliers = df[np.abs(z_scores) > threshold].copy()
    outliers['method'] = 'Z-Score'
    outliers['z_score'] = z_scores[np.abs(z_scores) > threshold]
    outliers['anomaly_type'] = np.where(outliers['z_score'] > 0, 'Peak (High)', 'Valley (Low)')
    return outliers

# --- 2. Data Preparation and Mode Manager (metric_desc update) ---

def get_mode_name(mode: str) -> str:
    """Converts the analysis mode code to a readable name."""
    mapping = {
        'TIME_AGGREGATED': 'Time-Series',
        'TIME_RAW': 'Time-Series',
        'DISTRIBUTIONAL': 'Distributional (Static)'
    }
    return mapping.get(mode.upper(), mode)

def get_freq_name(freq_code: str | None) -> str:
    """Converts frequency code (D, W, M) to readable name or 'N/A'."""
    if freq_code is None:
        return 'N/A'
    mapping = {'D': 'Daily', 'W': 'Weekly', 'M': 'Monthly'}
    return mapping.get(freq_code.upper(), freq_code)

def prepare_data_for_detection(
    df: pd.DataFrame, 
    value_col: str, 
    index_col: str,
    mode: str, 
    freq: str | None = None
) -> pd.DataFrame:
    """
    Manages data preparation based on the analysis mode, returning a standardized DataFrame.
    """
    df_copy = df.copy()
    
    # 1. Type Conversion
    if value_col not in df_copy.columns:
         raise ValueError(f"Value column '{value_col}' not found in DataFrame.")
    df_copy['value'] = df_copy[value_col].astype(float)
    
    # --- SCENARIO 1: DISTRIBUTIONAL (Static check) ---
    if mode.upper() == 'DISTRIBUTIONAL':
        ts_data = df_copy.rename(columns={index_col: 'index_id'})
        return ts_data[['index_id', 'value']].reset_index(drop=True)

    # --- SCENARIO 2 & 3: TIME-BASED ---
    
    if index_col not in df_copy.columns:
        raise ValueError(f"Time/Index column '{index_col}' not found for Time-based analysis.")
    df_copy['date'] = pd.to_datetime(df_copy[index_col])
    
    # --- SCENARIO 2: TIME_AGGREGATED ---
    if mode.upper() == 'TIME_AGGREGATED' and freq:
        agg_df = df_copy.groupby(pd.Grouper(key='date', freq=freq))['value'].sum().reset_index()
        agg_df.columns = ['date', 'value']
        agg_df['index_id'] = agg_df['date']
        return agg_df[agg_df['value'] > 0].reset_index(drop=True)

    # --- SCENARIO 3: TIME_RAW ---
    elif mode.upper() == 'TIME_RAW' or (mode.upper() == 'TIME_AGGREGATED' and not freq):
        ts_data = df_copy.rename(columns={index_col: 'index_id'})
        return ts_data[['index_id', 'value', 'date']][ts_data['value'] > 0].reset_index(drop=True)
    
    raise ValueError(f"Invalid mode '{mode}' or missing 'freq' for TIME_AGGREGATED mode.")


# --- 3. The Core Detection and Reporting (metric_desc update and improved print) ---

def anomaly_detection_core(
    ts_data: pd.DataFrame, 
    method: str, 
    mode_name: str, 
    freq_name: str,  # e.g., 'Daily', 'Weekly', 'Monthly', 'N/A'
    metric_desc: str 
) -> Dict:
    """
    Internal function to run detection and compile results using standardized data.
    """
    value_col = 'value'
    
    # ------------------ PRINT START ------------------
    print("-" * 50)
    print(f"Investigated Parameter: {metric_desc}")
    print(f"| Mode: {mode_name} | Frequency: {freq_name} | Method: {method}")
    print("-" * 50)

    # Handle empty/filtered data
    if ts_data.empty:
        print("-> Data is empty after preparation. Skipping detection.")
        return {
            "analysis_mode": mode_name,
            "frequency": freq_name,
            "metric_desc": metric_desc,
            "method": method,
            "total_points": 0,
            "anomaly_count": 0,
            "anomalies": [],
            "limit_description": "N/A - Empty Data"
        }

    # 1. Detect
    if method.upper() == 'IQR':
        outliers = detect_iqr_outliers(ts_data, value_col)

        # Determine IQR boundaries
        q1, q3 = ts_data[value_col].quantile(0.25), ts_data[value_col].quantile(0.75)
        iqr = q3 - q1
        lower_limit = q1 - (1.5 * iqr)
        upper_limit = q3 + (1.5 * iqr)
        limit_text = f"IQR Limits: {lower_limit:,.2f} to {upper_limit:,.2f}"

    else:
        outliers = detect_zscore_outliers(ts_data, value_col)
        limit_text = "Z-Score Limit: > +/- 3.0 Standard Deviations"

    num_outliers = len(outliers)
    print(f"-> Total Data Points: {len(ts_data)}")
    print(f"-> **Anomalies Found**: {num_outliers}")
    print(f"-> **Threshold**: {limit_text}")

    report_section = {
        "analysis_mode": mode_name,
        "frequency": freq_name,
        "metric_desc": metric_desc,
        "method": method,
        "limit_description": limit_text,
        "total_points": int(len(ts_data)),
        "anomaly_count": int(num_outliers),
        "anomalies": []
    }

    # ---------------- MANAGE OUTLIER LIST ----------------
    if not outliers.empty:
        print("\n--- ANOMALY DETAILS ---")

        outliers_to_report = outliers.sort_values(by=value_col, ascending=False)

        for _, row in outliers_to_report.iterrows():

            index_id_val = row['index_id']

            # FIXED: Conditional formatting for Timestamp-based indices
            if isinstance(index_id_val, pd.Timestamp):
                if freq_name == 'Monthly':
                    index_id_str = index_id_val.strftime('%Y-%m')          # e.g., 2023-11
                elif freq_name == 'Weekly':
                    index_id_str = index_id_val.strftime('%Y-W%W')        # e.g., 2023-W48
                else:
                    index_id_str = index_id_val.strftime('%Y-%m-%d')      # default
            else:
                index_id_str = str(index_id_val)

            val = row[value_col]

            # Determine method-specific reason text
            if method.upper() == 'IQR':
                if row['anomaly_type'] == 'Peak (High)':
                    info = "Breached Upper IQR Limit"
                else:
                    info = "Breached Lower IQR Limit"
            else:
                info = f"Z-Score: {row['z_score']:.2f}"

            # --- IMPROVED OUTPUT FORMAT ---
            print(f"[{index_id_str:<15}] | {row['anomaly_type']:<15} | "
                  f"Value: {val:,.2f} | Reason: {info}")

            report_section['anomalies'].append({
                "index_id": index_id_str,
                "value": float(val),
                "type": row['anomaly_type'],
                "method_details": info
            })

    else:
        print("\n-> **Result:** No anomalies detected based on the chosen threshold.")

    return report_section



# --- 4. The Unified Pipeline Runner (The single callable function - metric_desc update) ---

def perform_anomaly_detection(
    df: pd.DataFrame, 
    value_col: str,
    index_col: str,
    analysis_mode: str,
    metric_desc: str, # Renamed to metric_desc
    frequencies: List[str] = ['D', 'W'],
    method: str = 'IQR',
    output_path: Optional[Union[str, Path]] = None
) -> List[Dict]:
    """
    Runs the comprehensive anomaly detection pipeline across different modes.
    """
    full_report = []
    mode = analysis_mode.upper()
    mode_name = get_mode_name(mode)
    
    print("="*80)
    print(f"*** Anomaly Detection Analysis ***")

    # --- Mode Management ---
    if mode == 'TIME_AGGREGATED':
        if not frequencies:
             raise ValueError("frequencies list cannot be empty for TIME_AGGREGATED mode.")
             
        for freq in frequencies:
            ts_data = prepare_data_for_detection(df, value_col, index_col, mode, freq)
            report = anomaly_detection_core(ts_data, method, mode_name, get_freq_name(freq), metric_desc) 
            full_report.append(report)
            
    elif mode in ['TIME_RAW', 'DISTRIBUTIONAL']:
        ts_data = prepare_data_for_detection(df, value_col, index_col, mode)
        report = anomaly_detection_core(ts_data, method, mode_name, get_freq_name(None), metric_desc) 
        full_report.append(report)
        
    else:
        raise ValueError(f"Invalid analysis_mode specified: '{analysis_mode}'. Must be TIME_AGGREGATED, TIME_RAW, or DISTRIBUTIONAL.")
    
    print("\n" + "="*80)
    
    # Robust Saving Logic (metric_desc update)
    if output_path:
        try:
            path = Path(output_path)
            path.parent.mkdir(parents=True, exist_ok=True)
            with open(path, "w") as f:
                final_output = {
                    "pipeline_run_details": {
                        "metric_desc": metric_desc, 
                        "method": method,
                        "analysis_mode": mode_name,
                        "checks_run": [f"{r['frequency']} ({r['analysis_mode']})" for r in full_report]
                    },
                    "anomaly_checks": full_report
                }
                json.dump(final_output, f, indent=4)
            print(f"[Anomalies] Successfully saved report to -> {path}")
        except Exception as e:
            print(f"[ERROR] Could not save anomaly report to {output_path}. Error: {e}")
            
    return full_report

In [ ]:
perform_anomaly_detection(
    df=df, 
    value_col='payment_value',
    index_col='order_purchase_timestamp', 
    analysis_mode='TIME_AGGREGATED',
    metric_desc='Revenue',
    frequencies=  ['D', 'W', 'M'],
    method='Z-Score'
)

*** Anomaly Detection Analysis ***
--------------------------------------------------
Investigated Parameter: Revenue Kir Khar
| Mode: Time-Series | Frequency: Daily | Method: Z-Score
--------------------------------------------------
-> Total Data Points: 633
-> **Anomalies Found**: 2
-> **Threshold**: Z-Score Limit: > +/- 3.0 Standard Deviations

--- ANOMALY DETAILS ---
[2017-11-24     ] | Peak (High)     | Value: 179,200.03 | Reason: Z-Score: 10.18
[2017-11-25     ] | Peak (High)     | Value: 71,897.01 | Reason: Z-Score: 3.08
--------------------------------------------------
Investigated Parameter: Revenue Kir Khar
| Mode: Time-Series | Frequency: Weekly | Method: Z-Score
--------------------------------------------------
-> Total Data Points: 101
-> **Anomalies Found**: 1
-> **Threshold**: Z-Score Limit: > +/- 3.0 Standard Deviations

--- ANOMALY DETAILS ---
[2017-W47       ] | Peak (High)     | Value: 478,796.24 | Reason: Z-Score: 3.14
--------------------------------------------

C:\Users\rezam\AppData\Local\Temp\ipykernel_12252\313433839.py:94: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  agg_df = df_copy.groupby(pd.Grouper(key='date', freq=freq))['value'].sum().reset_index()


[{'analysis_mode': 'Time-Series',
  'frequency': 'Daily',
  'metric_desc': 'Revenue Kir Khar',
  'method': 'Z-Score',
  'limit_description': 'Z-Score Limit: > +/- 3.0 Standard Deviations',
  'total_points': 633,
  'anomaly_count': 2,
  'anomalies': [{'index_id': '2017-11-24',
    'value': 179200.03,
    'type': 'Peak (High)',
    'method_details': 'Z-Score: 10.18'},
   {'index_id': '2017-11-25',
    'value': 71897.01,
    'type': 'Peak (High)',
    'method_details': 'Z-Score: 3.08'}]},
 {'analysis_mode': 'Time-Series',
  'frequency': 'Weekly',
  'metric_desc': 'Revenue Kir Khar',
  'method': 'Z-Score',
  'limit_description': 'Z-Score Limit: > +/- 3.0 Standard Deviations',
  'total_points': 101,
  'anomaly_count': 1,
  'anomalies': [{'index_id': '2017-W47',
    'value': 478796.24,
    'type': 'Peak (High)',
    'method_details': 'Z-Score: 3.14'}]},
 {'analysis_mode': 'Time-Series',
  'frequency': 'Monthly',
  'metric_desc': 'Revenue Kir Khar',
  'method': 'Z-Score',
  'limit_description

In [46]:
perform_anomaly_detection(
    df=df, 
    value_col='payment_value',
    index_col='order_purchase_timestamp', 
    analysis_mode= 'DISTRIBUTIONAL',
    metric_desc='Order Value',
    frequencies= ['D'],
    method='IQR'
)

*** Anomaly Detection Analysis ***
--------------------------------------------------
Investigated Parameter: Order Value
| Mode: Distributional (Static) | Frequency: N/A | Method: IQR
--------------------------------------------------
-> Total Data Points: 99440
-> **Anomalies Found**: 7866
-> **Threshold**: IQR Limits: -110.43 to 349.41

--- ANOMALY DETAILS ---
[2017-09-29     ] | Peak (High)     | Value: 13,664.08 | Reason: Breached Upper IQR Limit
[2018-07-15     ] | Peak (High)     | Value: 7,274.88 | Reason: Breached Upper IQR Limit
[2017-02-12     ] | Peak (High)     | Value: 6,929.31 | Reason: Breached Upper IQR Limit
[2018-07-25     ] | Peak (High)     | Value: 6,922.21 | Reason: Breached Upper IQR Limit
[2017-05-24     ] | Peak (High)     | Value: 6,726.66 | Reason: Breached Upper IQR Limit
[2017-11-24     ] | Peak (High)     | Value: 6,081.54 | Reason: Breached Upper IQR Limit
[2017-04-01     ] | Peak (High)     | Value: 4,950.34 | Reason: Breached Upper IQR Limit
[2018-07-1

[{'analysis_mode': 'Distributional (Static)',
  'frequency': 'N/A',
  'metric_desc': 'Order Value',
  'method': 'IQR',
  'limit_description': 'IQR Limits: -110.43 to 349.41',
  'total_points': 99440,
  'anomaly_count': 7866,
  'anomalies': [{'index_id': '2017-09-29',
    'value': 13664.08,
    'type': 'Peak (High)',
    'method_details': 'Breached Upper IQR Limit'},
   {'index_id': '2018-07-15',
    'value': 7274.88,
    'type': 'Peak (High)',
    'method_details': 'Breached Upper IQR Limit'},
   {'index_id': '2017-02-12',
    'value': 6929.31,
    'type': 'Peak (High)',
    'method_details': 'Breached Upper IQR Limit'},
   {'index_id': '2018-07-25',
    'value': 6922.21,
    'type': 'Peak (High)',
    'method_details': 'Breached Upper IQR Limit'},
   {'index_id': '2017-05-24',
    'value': 6726.66,
    'type': 'Peak (High)',
    'method_details': 'Breached Upper IQR Limit'},
   {'index_id': '2017-11-24',
    'value': 6081.54,
    'type': 'Peak (High)',
    'method_details': 'Breached 